In [45]:
import sklearn.feature_extraction.text as txt
from sklearn import svm
from joblib import dump
from snowflake.snowpark import functions as fn
from snowflake.snowpark.session import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

In [ ]:
connection_parameters = {
    "account": "izoloqm-uv99170",
    "user": "xyz",
    "password": "xyz",
    "role": "accountadmin",
    "warehouse": "compute_wh",
    "database": "test",
    "schema": "public"
}

session = Session.builder.configs(connection_parameters).create()

In [47]:
import snowflake.snowpark.functions as F

In [48]:
df=session.table('orders')
df.schema

StructType([StructField('ORDER_ID', StringType(20), nullable=True), StructField('CUSTOMER_CODE', StringType(20), nullable=True), StructField('PLACED_AT', TimestampType(tz=ntz), nullable=True), StructField('RESTAURANT_ID', StringType(10), nullable=True), StructField('CUISINE', StringType(20), nullable=True), StructField('ORDER_STATUS', StringType(20), nullable=True), StructField('PROMO_CODE_NAME', StringType(20), nullable=True)])

In [49]:
df.show(5)

----------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"    |"CUSTOMER_CODE"   |"PLACED_AT"          |"RESTAURANT_ID"  |"CUISINE"  |"ORDER_STATUS"  |"PROMO_CODE_NAME"  |
----------------------------------------------------------------------------------------------------------------------------
|OF1900191801  |UFDDN1991918XUY1  |2025-01-01 15:30:20  |KMKMH6787        |Lebanese   |Delivered       |Tasty50            |
|OF1900191802  |UFDDN1991918XUY1  |2025-01-02 12:15:45  |LEBANESE2        |Lebanese   |Delivered       |NULL               |
|OF1900191803  |UFDDN1991918XUY1  |2025-01-10 18:45:30  |PIZZA123         |Italian    |Cancelled       |HUNGRY20           |
|OF1900191804  |UFDDN1991918XUY1  |2025-01-15 19:20:15  |ITALIAN2         |Italian    |Delivered       |NULL               |
|OF1900191805  |UFDDN1991918XUY1  |2025-01-20 11:30:00  |BURGER99         |American   |Delivered       |NULL               |


In [50]:
#task1:print the restaurant which has topmost orders with a specific cuisine without using limit and top
agg_df=df.group_by([F.col('RESTAURANT_ID'),F.col('CUISINE')]).agg(F.count('*').alias('total_orders')
                                                                  )
agg_df.show(10)

------------------------------------------------
|"RESTAURANT_ID"  |"CUISINE"  |"TOTAL_ORDERS"  |
------------------------------------------------
|KMKMH6787        |Lebanese   |10              |
|ITALIAN2         |Italian    |6               |
|AMERICAN2        |American   |6               |
|SUSHI456         |Japanese   |6               |
|TACO789          |Mexican    |7               |
|PIZZA123         |Italian    |10              |
|BURGER99         |American   |8               |
|LEBANESE2        |Lebanese   |9               |
|MEXICAN2         |Mexican    |6               |
|JAPANESE2        |Japanese   |5               |
------------------------------------------------



In [51]:
#create Window function
from snowflake.snowpark.functions import col, avg
from snowflake.snowpark import Window
window_orders = Window.partition_by([F.col('cuisine')]).order_by(F.col('total_orders').desc())
agg_df.select(F.col('restaurant_id'),F.col('cuisine'),F.col('total_orders'),F.row_number().over(window_orders).alias('orders_ranking')).filter(F.col('orders_ranking') == 1).show()

-------------------------------------------------------------------
|"RESTAURANT_ID"  |"CUISINE"  |"TOTAL_ORDERS"  |"ORDERS_RANKING"  |
-------------------------------------------------------------------
|BURGER99         |American   |8               |1                 |
|PIZZA123         |Italian    |10              |1                 |
|KMKMH6787        |Lebanese   |10              |1                 |
|TACO789          |Mexican    |7               |1                 |
|SUSHI456         |Japanese   |6               |1                 |
-------------------------------------------------------------------



In [52]:
#get the number of new customers each day at the outlet
from snowflake.snowpark.types import DateType
customer_first_order_date_df=df.with_column('order_date',F.cast(F.col('placed_at'),DateType())).group_by(F.col('customer_code')).agg(F.min(F.col('order_date')).alias('first_order_date'))
customer_first_order_date_df.group_by('first_order_date').agg(F.count('*').alias('total_new_customers')).order_by(F.col('first_order_date')).show()


----------------------------------------------
|"FIRST_ORDER_DATE"  |"TOTAL_NEW_CUSTOMERS"  |
----------------------------------------------
|2025-01-01          |2                      |
|2025-01-02          |1                      |
|2025-01-03          |1                      |
|2025-01-04          |1                      |
|2025-01-05          |3                      |
|2025-01-06          |1                      |
|2025-01-07          |1                      |
|2025-01-08          |1                      |
|2025-01-09          |1                      |
|2025-01-10          |3                      |
----------------------------------------------



In [53]:
customers_only_in_feb_march=[ i[0] for i in df.with_column('order_date',F.cast(F.col('placed_at'),DateType())).filter(F.month(F.col('order_date'))!=1  ).select(F.col('customer_code')).distinct().collect()]


In [54]:
customers_only_in_feb_march

['NO_ORDER_RECENT',
 'UVW7890123456JKL',
 'MULTI_CUISINE_CUST',
 'LAST_ORDER_7DAYS',
 'DEF9876543210XYZ',
 'UFDDN1991918XUY1',
 'NO_ORDER_LAST7_2',
 'ABC9876543210MNO',
 'CDE3456789012GHI',
 'NO_ORDER_LAST7_1',
 'ZZZ9876543210MNO',
 'PROMO_FIRST_ONLY']

In [55]:
df.with_column('order_date',F.cast(F.col('placed_at'),DateType())).filter(F.month(F.col('order_date'))==1 & (~F.col('customer_code').in_(customers_only_in_feb_march))  ).distinct().select(F.col('customer_code')).show()

--------------------
|"CUSTOMER_CODE"   |
--------------------
|GHI5678901234XYZ  |
|YZA3456789012ABC  |
|LMN9876543210JKL  |
|MNO7890123456XYZ  |
|VWX5678901234ABC  |
|SINGLE_ORDER_JAN  |
|BCD7890123456ABC  |
|ABC1234567890XYZ  |
|DEF5678901234MNO  |
|ABC1234567890XYZ  |
--------------------



In [57]:
customer_first_and_last_date=df.with_column('order_date',F.cast(F.col('placed_at'),DateType())).group_by(F.col('customer_code')).agg(F.min(F.col('order_date')).alias('first_order_date'),F.max(F.col('order_date')).alias('latest_order_date'))
customer_first_and_last_date.show()

---------------------------------------------------------------
|"CUSTOMER_CODE"   |"FIRST_ORDER_DATE"  |"LATEST_ORDER_DATE"  |
---------------------------------------------------------------
|PQR1234567890ABC  |2025-01-06          |2025-01-06           |
|QRS7890123456DEF  |2025-01-15          |2025-01-15           |
|CDE3456789012GHI  |2025-01-19          |2025-03-27           |
|TUV1234567890GHI  |2025-01-16          |2025-01-16           |
|SINGLE_ORDER_JAN  |2025-01-10          |2025-01-10           |
|GHI5678901234XYZ  |2025-01-03          |2025-01-03           |
|VWX5678901234ABC  |2025-01-08          |2025-01-08           |
|ZAB5678901234GHI  |2025-01-18          |2025-01-18           |
|PQR9876543210PQR  |2025-01-31          |2025-01-31           |
|JAN_ONLY_ORDER2   |2025-01-20          |2025-01-20           |
---------------------------------------------------------------



In [76]:
from datetime import datetime, timedelta
reference_date=datetime(2025,3,31)
customer_first_and_last_date.join(df.select(F.col('customer_code'),F.col('promo_code_name')),on='customer_code',how='inner').filter(
    (F.col('latest_order_date') <= reference_date - timedelta(days=7)) & (F.col('first_order_date') <= reference_date - timedelta(days=30)) 
    & (F.col('PROMO_CODE_NAME').is_not_null())).show(20)

------------------------------------------------------------------------------------
|"CUSTOMER_CODE"    |"FIRST_ORDER_DATE"  |"LATEST_ORDER_DATE"  |"PROMO_CODE_NAME"  |
------------------------------------------------------------------------------------
|ABC1234567890XYZ   |2025-01-01          |2025-01-05           |NEWUSER            |
|DEF9876543210XYZ   |2025-01-02          |2025-03-02           |FIRSTORDER         |
|GHI5678901234XYZ   |2025-01-03          |2025-01-03           |NEWUSER            |
|JKL3456789012XYZ   |2025-01-04          |2025-01-04           |FIRSTORDER         |
|PQR1234567890ABC   |2025-01-06          |2025-01-06           |NEWUSER            |
|VWX5678901234ABC   |2025-01-08          |2025-01-08           |FIRSTORDER         |
|BCD7890123456ABC   |2025-01-10          |2025-01-10           |NEWUSER            |
|HIJ9876543210DEF   |2025-01-12          |2025-01-12           |FIRSTORDER         |
|QRS7890123456DEF   |2025-01-15          |2025-01-15           |N

In [ ]:
#get customers if they place their 3rd order today thats  every 3rd order they will be communicated for discount

third_order_window=Window.partition_by(F.col('customer_code')).order_by(F.col('order_date'))
df.with_columns(['order_date','order_number'],[F.cast(F.col('placed_at'),DateType()),F.row_number().over(third_order_window)]).filter(
    (F.col('order_number')%3==0) & (F.col('order_date') == reference_date)).show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------
|"ORDER_ID"    |"CUSTOMER_CODE"     |"PLACED_AT"          |"RESTAURANT_ID"  |"CUISINE"  |"ORDER_STATUS"  |"PROMO_CODE_NAME"  |"ORDER_DATE"  |"ORDER_NUMBER"  |
--------------------------------------------------------------------------------------------------------------------------------------------------------------
|OF1900191870  |MULTI_CUISINE_CUST  |2025-03-31 14:45:00  |PIZZA123         |Italian    |Delivered       |NULL               |2025-03-31    |6               |
|OF1900191864  |LAST_ORDER_7DAYS    |2025-03-31 16:30:00  |KMKMH6787        |Lebanese   |Delivered       |NULL               |2025-03-31    |3               |
--------------------------------------------------------------------------------------------------------------------------------------------------------------



In [75]:
#customers who only order with promo code

df.group_by(F.col('customer_code')).agg(F.count('*').alias('total_number_of_orders'),
                                    F.count(F.col('promo_code_name')).alias('total_coupon_used')).filter((F.col('total_number_of_orders')==F.col('total_coupon_used'))
                                         & (F.col('total_number_of_orders') >1 )).show()

---------------------------------------------------------------------
|"CUSTOMER_CODE"   |"TOTAL_NUMBER_OF_ORDERS"  |"TOTAL_COUPON_USED"  |
---------------------------------------------------------------------
|DEF9876543210XYZ  |2                         |2                    |
|UVW7890123456JKL  |3                         |3                    |
---------------------------------------------------------------------



In [ ]:
#total customers in month of jan and total number of customers whose first order was without promocode
order_window=Window.partition_by(F.col('customer_code')).order_by(F.col('order_date'))
df.with_columns(['order_date','order_number'],[F.cast(F.col('placed_at'),DateType()),F.row_number().over(order_window)]).where(
    F.month(F.col('order_date')) == 1).select(F.sum(F.when((F.col('order_number') == 1) & 
                                                        (F.col('promo_code_name').is_not_null()),F.lit(1)).otherwise(F.lit(0))).alias('first_order_without_promocode'),F.count_distinct(F.col('customer_code')).alias('number_of_customers')).show()

-----------------------------------------------------------
|"FIRST_ORDER_WITHOUT_PROMOCODE"  |"NUMBER_OF_CUSTOMERS"  |
-----------------------------------------------------------
|23                               |41                     |
-----------------------------------------------------------



In [78]:
from snowflake.snowpark.functions import iif, col, lit


ImportError: cannot import name 'iif' from 'snowflake.snowpark.functions' (d:\anaconda\envs\snowflake_env\lib\site-packages\snowflake\snowpark\functions.py)